In [77]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime as dt
import copy
from sqlalchemy import create_engine, MetaData

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [2]:
#Đọc name map để chuyển đỏi các tên thành dạng full
name_map = pd.read_excel("data/t2m_classification.xlsx", sheet_name='name_map').drop(columns=['group'],axis=1)
name_map_dict = name_map.set_index('code')['full_name'].to_dict()

In [217]:
#Đọc toàn bộ các file csv được xuất ra từ ami itd
itd_item_dict = {}
folder_path = '../ami_itd_data'
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        key = os.path.splitext(filename)[0]
        itd_item_dict[key] = pd.read_csv(os.path.join(folder_path, filename)).sort_values('date', ascending=False).reset_index(drop=True)

for item, df in itd_item_dict.items():
    #Chuyển đổi về dạng m5
    df['minutes'] = df['date'].str.replace(' ', '').str.slice(4, 10).apply(lambda x: (x[:-1] + '0') if int(x[-1]) < 5 else (x[:-1] + '5'))
    df['date'] = df['date'].str.slice(0,7) + df['minutes'].str.slice(2) + "00"

    df = df.groupby(df['minutes']).agg({
            'date': 'last',
            'open': 'last',
            'high': 'max',
            'low': 'min',
            'close': 'first',
            'volume': 'sum'
        }).reset_index(drop=True)
    
    df['date'] = pd.to_datetime(df['date'].astype(str), format='%y%m%d %H%M%S')
    start_date = df['date'].max().normalize() - timedelta(days=6)
    df = df[(df['date'] > start_date)].reset_index(drop=True)

    itd_item_dict[item] = df

#Tạo bảng tổng hớp tất cả các item
itd_item_df = pd.DataFrame(list(itd_item_dict.keys())).rename(columns={0:'item'})
itd_item_df['len'] = itd_item_df['item'].apply(lambda x: len(x))
itd_item_df['last_2chars'] = itd_item_df['item'].str[-2:]
itd_item_df['third_last_char'] = itd_item_df['item'].str[-3:-2]
itd_item_df['first_4chars'] = itd_item_df['item'].str[:4]

#Lọc ra danh sách tên các cổ phiếu và index
stock_name_df = itd_item_df[itd_item_df['len']==3].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
index_name_df = itd_item_df[(itd_item_df['len']>3) & (itd_item_df['len']!=6) & (itd_item_df['len']<10) & (itd_item_df['item']!='0001')]\
                .reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)

In [219]:
#Đọc toàn bộ các file csv được xuất ra từ ami eod
eod_item_dict = {}
folder_path = '../ami_eod_data'
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        key = os.path.splitext(filename)[0]
        eod_item_dict[key] = pd.read_csv(os.path.join(folder_path, filename)).sort_values('date', ascending=False).reset_index(drop=True)

itd_stock_dict = {k:v for k,v in itd_item_dict.items() if k in stock_name_df['item'].tolist()}
itd_index_dict = {k:v for k,v in itd_item_dict.items() if k in index_name_df['item'].tolist()}
for key, df in itd_index_dict.items():
    start_date = df['date'].max().normalize()
    itd_index_dict[key] = df[(df['date'] > start_date)].reset_index(drop=True)

for item, df in eod_item_dict.items():
    df = df.loc[df['date'] > 200302]
    df['date'] = pd.to_datetime(df['date'].astype(str), format='%y%m%d')
    eod_item_dict[item] = df

#Tạo bảng tổng hớp tất cả các item
eod_item_df = pd.DataFrame(list(eod_item_dict.keys())).rename(columns={0:'item'})
eod_item_df['len'] = eod_item_df['item'].apply(lambda x: len(x))
eod_item_df['last_2chars'] = eod_item_df['item'].str[-2:]
eod_item_df['third_last_char'] = eod_item_df['item'].str[-3:-2]
eod_item_df['first_4chars'] = eod_item_df['item'].str[:4]

#Lọc ra danh sách tên các cổ phiếu và index
stock_name_df = eod_item_df[eod_item_df['len']==3].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
index_name_df = eod_item_df[(eod_item_df['len']>3) & (eod_item_df['len']!=6) & (eod_item_df['len']<10) & 
                (eod_item_df['third_last_char']!="F") & (eod_item_df['item']!='0001')].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
                
eod_stock_dict = {k:v.drop(['option'], axis=1) for k,v in eod_item_dict.items() if k in stock_name_df['item'].tolist()}
eod_index_dict = {k:v.rename(columns={'option':'value'}).drop('cap', axis=1)
                for k,v in eod_item_dict.items() if k in index_name_df['item'].tolist()}

#Lọc ra danh sách tên các cổ phiếu, index giao dịch tự doanh và nước ngoài
stock_name_td_df = eod_item_df[(eod_item_df['len']==6) & (eod_item_df['last_2chars']=='TD')].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
stock_name_nn_df = eod_item_df[(eod_item_df['len']==6) & (eod_item_df['last_2chars']=='NN')].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
index_td_nn_df = eod_item_df[(eod_item_df['len']>=10) & (eod_item_df['first_4chars']!='VN30') & ((eod_item_df['last_2chars']=='NN') | (eod_item_df['last_2chars']=='TD'))].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)

stock_td_dict = {k:v.drop(['high','low','cap'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in eod_item_dict.items() if k in stock_name_td_df['item'].tolist()}
stock_nn_dict = {k:v.drop(['high','low','cap'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in eod_item_dict.items() if k in stock_name_nn_df['item'].tolist()}
index_td_nn_dict = {k:v.drop(['high','low','cap','stock'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in eod_item_dict.items() if k in index_td_nn_df['item'].tolist()}

#Điều chỉnh đơn vị của các bảng NN và TD
for df in index_td_nn_dict.values():
    df['buy_volume'] = df['buy_volume']/1000
    df['sell_volume'] = -df['sell_volume']/1000
    df['buy_value'] = df['buy_value']/1000000000
    df['sell_value'] = -df['sell_value']/1000000000
    df['net_volume'] = df['buy_volume'] + df['sell_volume']
    df['net_value'] = df['buy_value'] + df['sell_value']

#Lọc ra danh sách các chỉ số phái sinh
index_name_fu_df = eod_item_df[(eod_item_df['len']>6) & (eod_item_df['third_last_char']=="F")].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
index_fu_dict = {k:v.drop(['cap'], axis=1).rename(columns={'option':'open_volume'})
                    for k,v in eod_item_dict.items() if k in index_name_fu_df['item'].tolist()}

#Lọc ra danh sách các chỉ số phái sinh khối ngoại
index_name_fu_nn_df = eod_item_df[(eod_item_df['len']>9) & (eod_item_df['first_4chars']=='VN30') & (eod_item_df['last_2chars']=='NN')].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
index_fu_nn_dict = {k:v.drop(['high','low','cap'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in eod_item_dict.items() if k in index_name_fu_nn_df['item'].tolist()}

#Lọc ra danh sách các chỉ số phái sinh tự doanh
index_name_fu_td_df = eod_item_df[(eod_item_df['len']>9) & (eod_item_df['first_4chars']=='VN30') & (eod_item_df['last_2chars']=='TD')].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
index_fu_td_dict = {k:v.drop(['high','low','cap'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in eod_item_dict.items() if k in index_name_fu_td_df['item'].tolist()}

#Tạo một date_series bao gồm khoảng ngày tính toán
date_series = pd.DataFrame(eod_index_dict['VNINDEX']['date']).rename(columns={0:'date'})